In [1]:
import cv2


In [2]:
img = cv2.imread("cards.jpg")
G = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
T = 165
T, B = cv2.threshold(G, thresh = T, maxval = 255, type = cv2.THRESH_BINARY)
cv2.namedWindow('Cards', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Cards', 800,950)
cv2.imshow("Cards", B)
key = cv2.waitKey(0)
cv2.destroyAllWindows()